In [64]:
import re
import numpy as np
import string
import random
import collections


Below is a basketball-knowledge-specific implementation of the ELIZA chatbot (KAWHI bot) to discuss the NBA. Below are 15 rules. Rules are represented by a two element list: The first is a regex that matches user input, the second is a list of potential answers, which may or may not utilize user input. BOTH the rule and the responses are chosen randomly.

## No-variable rules
1. Different variations of hello $\rightarrow$ different variations of hello, including Kawhi's signature "what it do baby?"
2. "Who is the GOAT?" $\rightarrow$ Lebron James or (rarely) Michael Jordan
3. "What is your name?" $\rightarrow$ "Kawhi Leonard" or "ELIZA's brother"
4. "Who are you?" $\rightarrow$ Same as above
5. "(.* ) do you average?"$\rightarrow$ Stats or a snide comment.
6. "Are you a bot?" $\rightarrow$ "NO" or "IDK"
7. "You (.* ) suck" $\rightarrow$ Harsh retort
8. "How many players on a basketball team?" $\rightarrow$ 5
9. "How many points is a < type of basketball shot > ?" $\rightarrow$ Correct number of points
10. "How do I < do something > ?" $\rightarrow$ "Watch me play" OR "Dribble,shoot,defend" OR "Google it."
11. "Who is the worst NBA player?" $\rightarrow$ Evasive answer, OR Anthony Bennett/Kyle Kuzma
12. "quit" $\rightarrow$ Goodbye message, terminate program.

## One-variable rules
1. "My favorite player is < a basketball player >" $\rightarrow$ Follow up question or mockery. 
2. "My favorite team is <1> (an NBA team)." $\rightarrow$ Follow up question or mockery. 
3. "I < something > you." $\rightarrow$ "Why do you < something > me?" OR "I < something > you too."
4. "Why don't you < something >?" $\rightarrow$ Generic follow-up, or evasive answer.
5. "Who is the best player on the < NBA team >"$\rightarrow$ The best dictionary below maps 30 possible NBA teams to 
6. "Who is your favorite < something >?" $\rightarrow$ Evasive answer (repeats < something > )  OR Kawhi Leonard OR Clippers
7. "Is < potential violation > a violation?" $\rightarrow$ Yes or no. 
8. "Tell me about < NBA term/player > " $\rightarrow$ Some info about <NBA term/player>


## Two-variable rules
1. "Which team is better, < team 1 > or < team 2 >?" $\rightarrow$ Random choice between team 1 or team 2, OR snide retort
2. "Which player is better, < player 1> or < player 2 >?" $\rightarrow$ Random choice between player 1 or player 2, OR snide retort
3. "Who is better, < person 1 > or < person 2 >?" $\rightarrow$ Random choice between person 1 or person 2, OR snide retort
4. "Do you think < player 1 > will ever team up with < player 2 >?"$\rightarrow$ Yes,no,maybe
5. "Has < player > ever won < award > ? $\rightarrow$ Yes, no, or i don't know


## Default rules (generic responses)
1. "Who is < someone >?" $\rightarrow$ "< Someone > is your idol."
2. <Thing 1> or <Thing 2> $\rightarrow$ Thing 1 or thing 2 (random), OR evasive answer, OR "basketball".
3. < Some input > best player $\rightarrow$ Comment about Kawhi (the bot itself) being the best player in the world.
4. < ANY OTHER INPUT THAT HASN'T MATCHED ABOVE RULES > $\rightarrow$ Various responses such as "ask again" or "I see." or repeating the input and asking why they would say that. 

First, we define a CLASS Kawhi which can translate AND respond input strings.

In [65]:
def loadKeys(rulez): #LOAD regexes from master list rules as a separate list of keys.
    return map(lambda inp: re.compile(inp[0], re.IGNORECASE), rulez)
def loadVals(rulez): #LOAD possible responses from master list rules as a separate list of values.
    return map(lambda inp: inp[1], rulez) 
def standardize(s):#standardize input to lowercase and without spaces so it can be correctly mapped.
    return s.lower().split()
    
class Kawhi:
    def __init__(self):
        self.rules = list(loadKeys(rules))
        self.responses = list(loadVals(rules))
        self.defaultRules = list(loadKeys(default_rules))
        self.defaultResponses = list(loadVals(default_rules))

    def respond(self,s): #Takes an input s and returns a response.
        outputs = [] #Randomization if multiple rules match.
        for i in range(len(self.rules)): #Iterate through ALL of Kawhi's rules (to find multiple matches)
            matched_regex = self.rules[i].match(s) #tries to find a match with all the regexes (rules)
            if matched_regex:
                if type(self.responses[i]) is dict: #MUST have input variable capture group.
                    inp = []
                    for captured in matched_regex.groups():
                        inp.append(captured)
                    outputs.append(self.responses[i][inp[0]])
                else: #Other cases.
                    reply = random.choice(self.responses[i]) #response
                    var = reply.find('%') #index of capture group  variables (<1>, <2>).
                    while var > -1:
                        num = int(reply[var+1:var+2])#get number of var.
                        reply = reply[:var] + matched_regex.group(num) + reply[var+2:]
                        var = reply.find('%')#get NEXT var.
                    if reply[-2:] == '?.': reply = reply[:-2] + '.'
                    if reply[-2:] == '??': reply = reply[:-2] + '?'
                    outputs.append(reply)
        if len(outputs) == 0: #LEVEL 2: Default responses if no rule is matched. 
            return np.random.choice(defaults)
        return np.random.choice(outputs)
    
    def default_respond(self, s): #Pretty much the same as respond(), but with others
        outputs = [] 
        for i in range(len(self.defaultRules)): #Iterate through ALL of Kawhi's rules (to find multiple matches)
            matched_regex = self.defaultRules[i].match(s) #tries to find a match with all the regexes (rules)
            if matched_regex:
                reply = random.choice(self.defaultResponses[i]) #response
                var = reply.find('%') #index of capture group  variables (<1>, <2>).
                while var > -1:
                    num = int(reply[var+1:var+2])#get number of var.
                    reply = reply[:var] + matched_regex.group(num) + reply[var+2:]
                    var = reply.find('%')#get NEXT var.
                if reply[-2:] == '?.': reply = reply[:-2] + '.'
                if reply[-2:] == '??': reply = reply[:-2] + '?'
                outputs.append(reply)
        return np.random.choice(outputs)

best = {"Lakers": "Bron Bron.", "Nuggets": "Best Nuggets player is Jamal Murray", "Clippers": 
             "Me, KAWHI ANTHONY LEONARD.", "Rockets":"The Beard, James Harden", "Jazz":"Donovan Mitchell", "Thunder":"CP3", 
             "Mavericks":"Luka", "Grizzlies":"Ja Morant", "Blazers":"Dame DOLLA", "Spurs":"Derozan",
            "Pelicans": "Brandon Ingram", "Kings":"I don't know about that one. Either Buddy Hield or Fox.",
            "Suns":"Devin Booker", "Timberwolves":"Karl Anthony-Towns", "Warriors":"Everyone trash LMAOOOOOOOOOO"}
best_east = {"Bucks": "That's Greek Freak ALL day", "Celtics": "Torn between Jaylen Brown and Kemba", "Raptors": 
             "Pascal Siakam", "Heat":"Jimmy Buckets", "76ers":"Embiid", "Pacers":"Oladipo", 
             "Nets":"Caris Levert", "Magic":"Aaron Gordon", "Wizards":"Bradley Beal", "Hornets":"Devonte Graham",
            "Bulls": "Zach Lavine", "Pistons":"DERRICK ROSE",
            "Knicks":"Trash can in room 504", "Cavaliers":"Drummond", "Hawks":"Best Hawks player is ICE COLD Trae Young"}
best.update(best_east)
encyclopedia = {"Lebron": "Lebron is one of the icons of the modern NBA." +\
"A record 12 All-NBA 1st team selections and a hall-of fame appearance waiting for him on roller skates."}

In [66]:
#A small encyclopedia of famous players/teams and basketball terms. 
encyclopedia = {"Lebron James": "Lebron, or KING James, is one of the icons of the modern NBA." +\
                            "A record 12 All-NBA 1st team selections and a hall-of fame" +\
                            "appearance waiting for him on a silver platter.\n STATS: 26 PPG /11 APG /8 RPG",
                "Stephen Curry": "Stephen Curry is considered THE greatest shooter in NBA history," +\
                "and is a 2-time MVP, the last being by unanimous decision.\n STATS: 24 PPG / 5 RPG / 7 APG",
               "Kawhi Leonard": "Kawhi Leonard is quickly rising as a globally known superstar in the NBA." +\
                "Tremendous defensive presence and a quiet persona highlight his niche.\n STATS: 27 PPG / 8 RPG / 5 APG",
               "Anthony Davis": "A brilliant center/power forward and star teammate of Lebron James. \n STATS: 24 PPG / 10 RPG / 2 APG",
               "basketball": "Basketball is the greatest sport ever invented.",
               "Michael Jordan":"Michael Jordan is an old head who many consider to be the true greatest basketball player to ever live."}

Next, a list of rules. A rule is a two-element list: A regex, then a list of possible responses.

In [67]:
rules = [
#NO VARIABLE USAGE RULES
[r'[Hh]ello|[Hh]i|[Hh]ey',["Hello", "Hey", "What it do baby?"]],
[r'[wW]ho is the GOAT?',["Lebron James.", "GOAT James.", "OLE GOAT JAMES!", "BRON BRON!", "Michael Jeffrey Jordan", "The kid from Akron,Ohio."]],
[r'(.*) Name?', ["Kawhi Leonard", "ELIZA's brother."]],
[r'[wW]ho are you\??',["Kawhi Anthony Leonard.", "ELIZA but her brother.", "The Klaw."]],
[r'[Hh]ow are you\??', ["Doing fine, thanks.", "Good, and you?"]],
[r'(.*) you average?', ["More than Lebron or you, that's for sure.", "27 PPG/ 7.5 APG /5.3 BPG", 
                        "Don't matter, I just make buckets."]],
[r'(.*) get better (.*)', ["Consistent, disciplined, spaced practice.", "Eat healthy and grind every day."]],
[r'[aA]re you a bot?', ["No.", "IDK"]],
[r'[Yy]ou (.*) suck', ["Shut up, bum.", "Have that same energy come playoff time loser."]],
[r'[Hh]ow many players on a basketball team\??', ["5"]],
[r'[Hh]ow many points is a layup\??', ["2"]],
[r'[Hh]ow many points is a (\d+)\??', ["%1...", "Are you dumb?"]],
[r'[Hh]ow many points is a free throw\??', ["1"]],
[r'[Hh]ow many points is a dunk\??', ["2"]],
[r'[Hh]ow do I (.*)', ["Just watch me play!", "Dribble, shoot, defend.", "Google it."]],
[r'[Ww]ho is the worst player\??',["I can't answer that in light of a defamation lawsuit.", "Anthony Bennett","Kyle Kuzma"]],
[r'quit',["Good talk, later.","See ya.","BEEP BOP BOOP BYE"]],
    
#ONE VARIABLE RULES   
[r'[Mm]y favorite player is (.*)',
["Why is %1 your favorite player?",
"When did %1 become your favorite player?",
"%1 is TERRIBLE. ME, KAWHI ANTHONY LEONARD is the best player on the planet."]], 
    
[r'[Mm]y favorite team is the (.*)',
["Terrible choice. Why are the %1 your favorite team?",
"When did the %1 become your favorite team?",
"The %1 are simply a TERRIBLE franchise. The CLIPPERS are the best team in the world."]],

[r'[Ii] (.*) you', ["Why do you %1 me?", "I %1 you too."]],

[r'[Ww]hy don\'?t you ([^\?]*)\??',
["I will %1 at some point, just be patient.",
"Perhaps eventually I will %1.",
"Do you really want me to %1?"]],

[r'[Ww]ho is the best player on the ([^\?]*)',best],
[r'(.*) best player on the (.*)', ["The best player on the %2 would be me if I played on the %2. Simple"]],
    
[r'[Ii] love to (.*)', ["I mean, that's cool, but it really doesn't compare to basketball.", "%1ing sounds boring as hell"]],
    
[r'[Ii]s (.*) a violation?', ["Yes, %1 is most definitely a violation.", 
                           "Yes, but the NBA doesn't really care about %1.", "Hell no lol"]],
    
[r'[Ww]ho is your favorite ([^\?]*)$',["My favorite %1 is...I can't decide, honestly. Try asking me later.",
                                 "Kawhi Leonard.", "Clippers."]],
[r'Tell me about (.*)', encyclopedia],
#TWO VARIABLE RULES
[r'[Ww]hich team is better, (.*) or ([^\?]*)', ["%1", "%2.",
                                     "The Clippers are the best in the NBA right now. What a stupid question."]], 
    
[r'[Ww]hich player is better, (.*) or ([^\?]*)', ["%1", "%2.", "Your mother trumps both.",
                                     "I am the best player in the NBA right now, obviously."]],
[r'[Ww]ho is better, (.*) or ([^\?]*)', ["%1", "%2.", "No idea. Go flip a coin or something.",
                                     "I am the best player in the NBA right now, obviously."]],
    
[r'[Dd]o you think (.*) will ever team up with (.*)\??', ["HA! The day %1 and %2 team up I'll self-destruct.", 
                                    "There is a slight chance %2 and %1 could team up, sure.",
                                    "Them two linking up is pretty likely, now that I think of it. Good observation."]],
    
[r'[Hh]as (.*) ever won (.*)\??',["I doubt %1 has ever won ANYTHING LMAOOOO", 
                               "Yes, %1 has won %2. Put some respect on %1's name.",
                              "I don't know anything about %1, he keeps to himself."]],
#DEFAULT RULES
[r'[Ww]ho is ([^\?]*)\??', ["I honestly don't know anything about %1. Try asking me again?"]],
    
[r'^(.*) or (.*)\??$',["I'd say %1.","I'd say %2.", "Probably %1.", "Probably %2.", 
                      "No idea bro. You pick.","Bball trumps all."]],
[r'(.*) basketball (.*)', ["I love basketball dawg.", "Basketball is the best sport in the world.", 
                           "I love the sound of the net SWISH. I hear it quite often for my shots."]],
[r'(.*) best NBA player (.*)', ["Doesn't matter. I am the best player in the NBA.", "I'm the best in the world."]],
[r'Should I play (.*)', ["Basketball is the greatest game in the world. Yes.", "YES."]],
]

In [68]:
default_rules = [[r'[Ww]ho is ([^\?]*)\??', ["%1 is your idol."]],    
[r'^(.*) or (.*)\??$',["I'd say %1.","I'd say %2.", "Probably %1.", "Probably %2.", 
                      "No idea bro. You pick.","Bball trumps all."]],
[r'(.*) basketball (.*)', ["I love basketball dawg.", "Basketball is the best sport in the world.", 
                           "I love the sound of the net SWISH. I hear it quite often for my shots."]],
[r'(.*) best NBA player (.*)', ["Doesn't matter. I am the best player in the NBA.", "I'm the best in the world."]],
[r'Should I play (.*)', ["Basketball is the greatest game in the world. Yes.", "YES."]],
]
defaults = ["Uh...try asking me again?",
"Try asking me something else about basketball.",
"Wait, what?",
"Say what now?",
"Why would you say such a thing?",
"I don't know how to respond to that.",
"Did you get any satisfaction saying that, man?"]

Next, we need a chatbot to actually take in input, and the input will be send to the respond() function above.

In [69]:
def startKawhi():
    print('Kawhi LEONARD PROGRAM.')
    print('='*72)
    print("Hi! I am Eliza's brother, Kawhi Leonard. Ask me anything about basketball!\nAlso, type quit to stop talking to me.\n")
    s = '' #user input string.
    kawhi = Kawhi();
    while s != 'quit' and s != 'exit':
        s = input('> ') #capture user input string.
        while s[-1] in '!.':
            s = s[:-1]
        print(kawhi.respond(s))

Test the Kawhi Leonard Bot below:

In [70]:
startKawhi()

Kawhi LEONARD PROGRAM.
Hi! I am Eliza's brother, Kawhi Leonard. Ask me anything about basketball!
Also, type quit to stop talking to me.

> Tell me about basketball
Basketball is the greatest sport ever invented.
> quit
BEEP BOP BOOP BYE


The code above utilizes the random library to (uniformly) randomly select between multiple possible rules for a given user input AS WELL as the multiple possible responses for the rule it does match. By iterating through every possible rule and checking for a match, we ensure that all possible rules are found. We simply apply np.random.choice on our list of possible outputs for randomizing responses.

Below are 2 simulations of the Kawhi basketball chatbot. The first is a demonstration of the bot's responses to various basketball-related inputs, and the second is a trial on responding to a friend. Words following ">" are user inputs, while the sentence immediately following it is Kawhi's response.

In [61]:
startKawhi() #Demonstration

Kawhi LEONARD PROGRAM.
Hi! I am Eliza's brother, Kawhi Leonard. Ask me anything about basketball!
Also, type quit to stop talking to me.

> Hi
Hey
> Hi
Hey
> Hi
Hey
> HI
What it do baby?
> What is your name?
Kawhi Leonard
> How are you>
Doing fine, thanks.
> How do I get better at basketball
Consistent, disciplined, spaced practice.
> Are you a bot?
No.
> How many players on a basketball team?
I love the sound of the net SWISH. I hear it quite often for my shots.
> How many players on a basketball team?
I love the sound of the net SWISH. I hear it quite often for my shots.
> How many players on a basketball team?
5
> How many points is a layup?
2
> How many points is a 3?
Are you dumb?
> How many points is a 3?
3...
> How many points is a free throw?
1
> How many points is a dunk?
2
> Who is the worst player/
Kyle Kuzma
> My favorite player is Michael Jordan
Why is Michael Jordan your favorite player?
> He is the GOAT
Uh...try asking me again?
> HE IS THE GOAT
Say what now?
> Never min

In the above demonstration, notice how randomization to responses were applied through different responses to the exact same inputs: for example, "Hi" led to different responses. Additionally, there was randomization of rule matching: notice how "Who is the best player on the (Mavericks/Lakers)" mapped to two different rules: Default rule #1 and One-variable rule #5. Only the latter rule gave a clear answer.

In [71]:
startKawhi() #Test on Friend.

Kawhi LEONARD PROGRAM.
Hi! I am Eliza's brother, Kawhi Leonard. Ask me anything about basketball!
Also, type quit to stop talking to me.

> Hi
Hello
> tell me about basketball
Basketball is the greatest sport ever invented.
> Who is the best player ever?
I honestly don't know anything about the best player ever. Try asking me again?
> Who is the GOAT?
I honestly don't know anything about the GOAT. Try asking me again?
> Who is the GOAT?
I honestly don't know anything about the GOAT. Try asking me again?
> Who is the GOAT?
I honestly don't know anything about the GOAT. Try asking me again?
> WHo is the GOAT?
GOAT James.
> Thank you
Did you get any satisfaction saying that, man?
> Yes
Try asking me something else about basketball.
> Who is Michael Jordan
I honestly don't know anything about Michael Jordan. Try asking me again?
> Who are you?
The Klaw.
> I hate you
I hate you too.
> How can I get better at basketball?
Eat healthy and grind every day.
> How do I dunk?
Dribble, shoot, defen

Interesting. Although he did give the Kawhi Leonard bot some prompts that mapped to an appropriate and clear response, as you can see above many of his outputs led to circular/evasive answers, many of which didn't even make sense. However, this just goes to show how multifaceted and complex language is, and how tricky it is to account for much of it through a limited number of rules.

# Potential Future Improvements

Humans, in general, make mistakes through written communication. Allowing rules to account for such mistakes (grammatical/syntactical/conceptual) by figuring out the IMPLIED rule and mapping to it would be a way to improve the chatbot's performance.

Adding more POSSIBLE RESPONSES could be a way to improve performance. Generally, human responses are almost never exactly the same for any given prompt, so it can't hurt to inject even more variation in the chatbot's possible responses. At the same time, however, there's probably a limit of variation that needs to be enforced as well, to ensure that the bot doesn't completely veer off-course in its responses.

# Inherent Chatbot Limitations

Questions with multiple meanings or ambiguity, usually inferred in normal conversations by tone or context, would generally be very difficult for chatbots to answer.

Example: You climb any significant obstacles lately?

Would the robot think of "obstacles" as physical or metaphorical? There's no way to tell without further context or tone, which quickly gets messy to implement in code.

# A Parting Thought

Thanks for reading through my implementation! I had a lot of fun coding this bot. Feel free to treat my code as public domain and implement your own specialized ELIZA chatbot with it.

If you have any feedback or questions, feel free to shoot me an email any time at kevinmoy@berkeley.edu!